In [1]:
import numpy as np
from gurobipy import *
import pandas as pd
np.random.seed(1)

In [2]:
def Corte_Entrada(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [(x,y)]
    return Ce

def Corte_Salida(i):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [(x,y)]
    return Cs

def Corte_Entrada_Nodo(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [x]
    return Ce

def Corte_Salida_Nodo(i,A):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [y]
    return Cs


## PRIMERA FASE HEURÍSTICA

In [3]:
def Auto(gap =0, time =60):
    m = Model()
    
    x = m.addVars(L, name="x", vtype=GRB.INTEGER, lb = 0)
    z = m.addVars(L, name="z", vtype=GRB.BINARY)
    
    m.update()
    
    #Función Objetivo
    t_V = {(k,m,p): c_M[k,m] for (k,m,p) in L}
    m.setObjective(z.prod(t_V) , GRB.MINIMIZE)
    
     #Restricciones
    m.addConstrs(
       (Dem[p] == x.sum("*","*","D"+str(p)) for p in range(1,n2+1))
    )
    
    m.addConstrs(
        (x[k,m,p] <= Q[k,m]*z[k,m,p] for (k,m,p) in L)
    )
    
    m.addConstrs(
        (z.sum(k,m,"*") <= 1 for k in range(1,n1+1) for m in M[k])
    )
    
    m.Params.MIPGap = gap
    m.Params.TimeLimit= time
    m.optimize()
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}
    
    
    Autos = {}
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        for (k,m1,d) in L:
             if x[k,m1,d].x>0:
                    Autos.update({(k,m1,d):x[k,m1,d].x })
                    print(" De {} el auto  {} va a {} y aporta {}".format(k,m1,d,x[k,m1,d].x))
    
    
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        for (k,m1,d) in L:
             if z[k,m1,d].x>0:
                    print(" De {} el auto  {} va a {}se usa".format(k,m1,d))
    
    return Autos, par

## SEGUNDA FASE HEURÍSTICA

In [8]:
def Heuritic(gap=0,time=3600):
    m = Model()
    
    # Crear las variables
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for (k, m, j) in R], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i) for (k,m) in Autos.keys() for i in range(1,n+1)] , name = "a", vtype = GRB.INTEGER, lb=0)
    
    u = m.addVars( V, name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()

    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}
    
    m.setObjective(x.prod(t_V) + costo  , GRB.MINIMIZE)
    
    #Restricciones
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[j]  >= u[i] + (1 + n)*x[k,b,i,j] - n)
        )
        
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  == 1 for (k,m) in Autos.keys()) 
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*", Autos[k,m][0])  == 1 for (k,m) in Autos.keys()) 
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i] for (k,m) in Autos.keys() for i in range(1,n+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for (k,m) in Autos.keys() for s in V)
    )
    
    m.addConstrs(
        (a.sum("*","*",i) <= OF[i] for i in V)
    )
    
    m.addConstrs(
        (a.sum(k,m,"*") == Autos[k,m][1] for (k,m) in Autos.keys())
    )
    
    m.addConstrs(
        (a[k,m,i] <= OF[i]*y[k,m,i] for (k,m) in Autos.keys() for i in range(1,n+1) )
    )
    
    
    m.Params.MIPGap = gap
    m.Params.TimeLimit= time
    m.optimize()
    
    
    
    
    
    clasi = {}
    par={}
    producto = {}
    
    try:
        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}
        # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            for (k,m1,i,j) in K:
                if vx[k,m1,i,j] >= 0.99:
                    print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                    if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                        clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                    else:
                        clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]




        print("-"*20)

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for (k,m1) in Autos.keys():
                for i in range(1,n+1):
                    if a[k,m1,i].x>0:
                        print("El auto {} del depósito {} recoge {} en {}".format(m1,k,a[k,m1,i].x,i))
                        if ("O"+str(k)+"-"+str(m1) in producto.keys() )== False:
                            producto.update({"O"+str(k)+"-"+str(m1) : [(i,a[k,m1,i].x,Autos[k,m1][0])]})
                        else:
                            producto["O"+str(k)+"-"+str(m1)]  = producto["O"+str(k)+"-"+str(m1)] +  [(i,a[k,m1,i].x,Autos[k,m1][0])]


        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for (k,m1,j) in R:
                if y[k,m1,j].x>0.99:
                    print("el auto {} del deposito {} visita {}".format(m1, k, j))
    except:
        print("TIME OUT!")
    
    return clasi, producto, par
    
    

## LECTURA Y RESOLUCIÓN CON LA HEURÍSICA DE LAS 52 INTANCIAS 

In [17]:
import pandas as pd
from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal - Copy.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {5: {11: [17], 14: [17], 17: [2], 18: [6]},
 10: {11: [10], 15: [4], 17: [9], 21: [11]},
 15: {6: [5], 7: [6], 18: [7], 22: [9]},
 20: {12: [2], 13: [8], 19: [10], 24: [4]},
 25: {11: [12], 16: [9], 17: [17], 24: [17]},
 30: {10: [2], 15: [3], 20: [12], 24: [16]},
 35: {6: [15], 15: [2], 19: [5], 23: [19]},
 40: {6: [4], 7: [8], 13: [8], 15: [15]},
 45: {14: [13], 19: [9], 23: [14], 25: [15]},
 50: {13: [13], 14: [14], 16: [13], 21: [10]},
 55: {9: [6], 12: [9], 23: [9], 24: [15]},
 60: {10: [6], 17: [20], 18: [2], 23: [18]},
 65: {15: [9], 19: [12], 22: [15], 25: [19]}}

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias4 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }
                        
           
            
            print("AUTOS HEURISTICA")
            L = [  (k,m,p) for k in range(1,n1+1) for m in M[k] for p in D ]
            Au = Auto()
            costo = Au[-1]["Obj"]
            tiempo = Au[-1]["Time"]
            Au = Au[0]
            
            #Autos
            Autos = { (k,m): (d,Au[(k,m,d)]) for(k,m,d) in Au.keys() }
            Autos

            #Crear K Para X

            X_D = [(k, m, i, d) for (k,m,d) in Au.keys()  for i in Corte_Entrada_Nodo(d,Ad)   ]
            X_A = [(k, m, i, j) for (k,m) in Autos.keys()  for (i,j) in A   ]
            X_O = [(k, m, j[0], j[1]) for (k,m) in Autos.keys() for j in Ao if "O"+str(k) == j[0]  ]
            K = X_O + X_A + X_D

            #Crear R Para y

            R = [(k,m,d) for (k,m,d) in Au.keys()  ] + [(k,m,i) for (k,m) in Autos.keys() for i in range(1,n+1)]
            
            
            print("HEURISTICA ASIGANCIÓN")
            Ruta, Pro, Par =  Heuritic(time=600)
            
            Par["Time"] = Par["Time"] + tiempo
            
            DatosInstancias4.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta, Pro, Par] })

<span style="color: #ff0000">5,11 y 17</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 683 rows, 1258 columns and 2516 nonzeros
Model fingerprint: 0xd39f7f0a
Variable types: 0 continuous, 1258 integer (629 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.01s
Presolved: 683 rows, 1258 columns, 2516 nonzeros
Variable types: 0 continuous, 1258 integer (629 binary)

Root relaxation: objective 7.847620e+03, 749 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7847.61983    0   21          - 7847.61983      -     -    0s
H    0     0                    1

<span style="color: #ff0000">5,14 y 17</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 683 rows, 1258 columns and 2516 nonzeros
Model fingerprint: 0x83939ad6
Variable types: 0 continuous, 1258 integer (629 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.01s
Presolved: 683 rows, 1258 columns, 2516 nonzeros
Variable types: 0 continuous, 1258 integer (629 binary)

Root relaxation: objective 9.525505e+03, 763 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9525.50505    0   23          - 9525.50505      -     -    0s
H    0     0                    1

<span style="color: #ff0000">5,17 y 2</span> 

...

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 559 rows, 1020 columns and 2040 nonzeros
Model fingerprint: 0xbc36b7f3
Variable types: 0 continuous, 1020 integer (510 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 559 rows, 1020 columns, 2040 nonzeros
Variable types: 0 continuous, 1020 integer (510 binary)

Root relaxation: objective 8.058760e+03, 604 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8058.75962    0   20          - 8058.75962      -     -    0s
H    0     0                    1

     0     0 16880.0606    0 1162          - 16880.0606      -     -  109s
     0     0 16880.0754    0 1208          - 16880.0754      -     -  110s
     0     0 16880.2110    0 1188          - 16880.2110      -     -  114s
     0     0 16880.2110    0 1188          - 16880.2110      -     -  116s
     0     2 16880.2110    0 1188          - 16880.2110      -     -  147s
     1     4 16883.3593    1 1060          - 16880.2164      -   720  151s
     7    12 16886.0568    3  969          - 16883.3485      -   491  157s
    11    16 16886.0177    3  949          - 16883.3485      -   562  160s
    19    24 16886.7398    4  905          - 16884.5031      -   620  166s
    27    32 16886.9406    5  922          - 16884.7826      -   583  173s
    31    36 16887.9765    6  918          - 16884.9782      -   591  175s
    40    45 16893.9539    7  843          - 16885.7629      -   570  181s
    44    49 16893.9235    8  903          - 16885.7635      -   568  189s
    48    54 16891.2839  

<span style="color: #ff0000">65,25 y 19</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1179 rows, 2204 columns and 4408 nonzeros
Model fingerprint: 0xa3eae584
Variable types: 0 continuous, 2204 integer (1102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.02s
Presolved: 1179 rows, 2204 columns, 4408 nonzeros
Variable types: 0 continuous, 2204 integer (1102 binary)

Root relaxation: objective 9.081078e+03, 1258 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9081.07767    0   25          - 9081.07767      -     -    0s
H    0     0                

     0     0 20008.1662    0 1337          - 20008.1662      -     -   89s
     0     0 20008.1854    0 1371          - 20008.1854      -     -   90s
     0     0 20008.5330    0 1416          - 20008.5330      -     -   94s
     0     0 20008.6315    0 1425          - 20008.6315      -     -   95s
     0     0 20008.6523    0 1467          - 20008.6523      -     -   96s
     0     0 20008.9807    0 1443          - 20008.9807      -     -   99s
     0     0 20009.3248    0 1402          - 20009.3248      -     -  100s
     0     0 20009.8225    0 1400          - 20009.8225      -     -  102s
     0     0 20009.8389    0 1456          - 20009.8389      -     -  103s
     0     0 20010.1275    0 1434          - 20010.1275      -     -  106s
     0     0 20010.1863    0 1422          - 20010.1863      -     -  107s
     0     0 20010.2129    0 1434          - 20010.2129      -     -  108s
     0     0 20010.5780    0 1530          - 20010.5780      -     -  111s
     0     0 20010.7054  

## INTANCIAS A LAS CUALES EN 600 SEGUNDOS NO ALCANZARON UNA SOLUCIÓN ÓPTIMA

In [19]:
import pandas as pd
from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal - Copy.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {
 35: {6: [15], 15: [2]},
 50: {16: [13]},
 55: {24: [15]},
 }

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias5 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }
                        
           
            
            print("AUTOS HEURISTICA")
            L = [  (k,m,p) for k in range(1,n1+1) for m in M[k] for p in D ]
            Au = Auto()
            costo = Au[-1]["Obj"]
            tiempo = Au[-1]["Time"]
            Au = Au[0]
            
            #Autos
            Autos = { (k,m): (d,Au[(k,m,d)]) for(k,m,d) in Au.keys() }
            Autos

            #Crear K Para X

            X_D = [(k, m, i, d) for (k,m,d) in Au.keys()  for i in Corte_Entrada_Nodo(d,Ad)   ]
            X_A = [(k, m, i, j) for (k,m) in Autos.keys()  for (i,j) in A   ]
            X_O = [(k, m, j[0], j[1]) for (k,m) in Autos.keys() for j in Ao if "O"+str(k) == j[0]  ]
            K = X_O + X_A + X_D

            #Crear R Para y

            R = [(k,m,d) for (k,m,d) in Au.keys()  ] + [(k,m,i) for (k,m) in Autos.keys() for i in range(1,n+1)]
            
            
            print("HEURISTICA ASIGANCIÓN")
            Ruta, Pro, Par =  Heuritic(time=900)
            
            Par["Time"] = Par["Time"] + tiempo
            
            DatosInstancias5.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta, Pro, Par] })

<span style="color: #ff0000">35,6 y 15</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 431 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xfa3606cb
Variable types: 0 continuous, 780 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 10694.000000
Presolve time: 0.00s
Presolved: 431 rows, 780 columns, 1560 nonzeros
Variable types: 0 continuous, 780 integer (390 binary)

Root relaxation: objective 6.342015e+03, 485 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6342.01527    0    9 10694.0000 6342.01527  40.7% 

     0     0 12695.7721    0  732 12870.0000 12695.7721  1.35%     -   19s
     0     0 12695.8349    0  762 12870.0000 12695.8349  1.35%     -   19s
     0     0 12695.8677    0  728 12870.0000 12695.8677  1.35%     -   19s
     0     0 12695.8848    0  744 12870.0000 12695.8848  1.35%     -   19s
     0     0 12695.8980    0  715 12870.0000 12695.8980  1.35%     -   19s
     0     0 12695.9008    0  731 12870.0000 12695.9008  1.35%     -   19s
     0     0 12696.0084    0  718 12870.0000 12696.0084  1.35%     -   20s
     0     0 12696.0335    0  682 12870.0000 12696.0335  1.35%     -   20s
     0     0 12696.0668    0  736 12870.0000 12696.0668  1.35%     -   20s
     0     0 12696.1028    0  725 12870.0000 12696.1028  1.35%     -   20s
     0     0 12696.1116    0  766 12870.0000 12696.1116  1.35%     -   20s
     0     0 12696.1743    0  769 12870.0000 12696.1743  1.35%     -   21s
     0     0 12696.2012    0  713 12870.0000 12696.2012  1.35%     -   21s
     0     0 12696.2164  

  4332  2083 12769.9658   42  816 12801.0000 12765.1798  0.28%   262  508s
  4506  2164 12771.3830   46  731 12801.0000 12765.2392  0.28%   263  516s
  4681  2252 12771.0621   47  687 12801.0000 12765.5265  0.28%   265  527s
  4827  2330 12789.3320   48  654 12801.0000 12765.7728  0.28%   268  537s
  4987  2485 12778.4634   47  608 12801.0000 12765.9130  0.27%   271  549s
  5232  2517 infeasible   57      12801.0000 12766.2724  0.27%   269  562s
  5304  2617 12773.9003   43  832 12801.0000 12766.3695  0.27%   271  574s
  5469  2731 12787.9216   54  700 12801.0000 12766.7872  0.27%   274  586s
  5694  2807 12775.2509   47  748 12801.0000 12767.4202  0.26%   275  600s
  5891  2925 12790.4672   61  488 12801.0000 12767.7508  0.26%   275  612s
  6113  3068 12791.2182   59  478 12801.0000 12768.1047  0.26%   277  626s
  6348  3188 12780.1441   57  560 12801.0000 12768.2435  0.26%   278  641s
  6601  3381 12787.1150   46  654 12801.0000 12768.6002  0.25%   280  658s
  6902  3502 12777.6880  

<span style="color: #ff0000">35,15 y 2</span> 

...

     0     0 14941.0852    0 1114          - 14941.0852      -     -   51s
     0     0 14941.0884    0 1133          - 14941.0884      -     -   51s
     0     0 14941.1730    0 1055          - 14941.1730      -     -   52s
     0     0 14941.2853    0 1028          - 14941.2853      -     -   53s
     0     0 14941.3210    0 1046          - 14941.3210      -     -   54s
     0     0 14941.3369    0 1093          - 14941.3369      -     -   54s
     0     0 14941.3403    0 1105          - 14941.3403      -     -   54s
     0     0 14941.5097    0 1069          - 14941.5097      -     -   55s
     0     0 14941.5525    0 1158          - 14941.5525      -     -   56s
     0     0 14941.5620    0 1175          - 14941.5620      -     -   56s
     0     0 14942.0599    0  912          - 14942.0599      -     -   58s
     0     0 14942.1175    0  932          - 14942.1175      -     -   58s
     0     0 14942.1968    0  961          - 14942.1968      -     -   59s
     0     0 14942.2056  

  4122  3833 15056.1629   94 1330          - 14961.3331      -   225  688s
  4124  3834 17458.1538  733 1483          - 14961.5320      -   225  690s
  4128  3837 15047.5133   78 1524          - 14961.7291      -   225  695s
  4129  3837 15504.7328  289 1431          - 14962.2782      -   225  703s
H 4130  3644                    16856.000000 14962.4657  11.2%   225  704s
  4132  3645 15711.1166  411 1488 16856.0000 14962.7069  11.2%   225  705s
H 4133  3463                    16809.000000 14962.7420  11.0%   225  711s
H 4134  3289                    16715.000000 14963.4332  10.5%   225  719s
  4136  3290 14968.9447   12 1460 16715.0000 14964.0102  10.5%   225  721s
  4142  3294 16715.0000  717 1629 16715.0000 14964.4745  10.5%   224  730s
H 4142  3128                    15559.000000 14964.8943  3.82%   224  784s
  4144  3129 15086.4521   98 1509 15559.0000 14965.1863  3.82%   224  785s
  4149  3133 15559.0000  492 1630 15559.0000 14965.4733  3.81%   224  791s
  4150  3133 15559.0000  

<span style="color: #ff0000">55,24 y 15</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 591 rows, 1080 columns and 2160 nonzeros
Model fingerprint: 0xf7ab7dba
Variable types: 0 continuous, 1080 integer (540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 13027.000000
Presolve time: 0.01s
Presolved: 591 rows, 1080 columns, 2160 nonzeros
Variable types: 0 continuous, 1080 integer (540 binary)

Root relaxation: objective 8.411263e+03, 653 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8411.26316    0   23 13027.0000 8411.26316  35

   151   164 16559.7005   31  217          - 16540.9815      -   361  156s
   163   174 16561.5994   35  266          - 16540.9815      -   357  160s
   186   204 16566.7296   39  188          - 16540.9815      -   363  166s
   205   224 16567.0969   43  210          - 16540.9815      -   355  170s
   239   260 16574.1718   54  223          - 16540.9815      -   360  178s
   261   273 16576.7728   59  269          - 16540.9815      -   347  182s
   274   295 16577.9917   61  200          - 16540.9815      -   348  187s
   296   310 16581.5933   70  206          - 16540.9815      -   343  191s
   311   333 16585.0741   75  232          - 16540.9815      -   346  196s
   334   353 16585.3540   77  308          - 16540.9815      -   341  201s
   356   373 16587.7345   80  299          - 16540.9815      -   340  206s
   376   397 16589.8439   85  269          - 16540.9815      -   342  211s
   400   423 16597.7740   90  263          - 16540.9815      -   345  216s
   427   448 16599.5541  

In [28]:
for i in DatosInstancias5.keys():
    DatosInstancias4[i] = DatosInstancias5[i]

## INTANCIAS A LAS CUALES NO SE OBTUVO SOLUCIÓN FACTIBLE NI EN 600SEGUNDOS NI EN 900SEGUNDOS

In [22]:
import pandas as pd
from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal - Copy.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {
 60: {23: [18]},
 65: {15: [9]}}

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias6 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }
                        
           
            
            print("AUTOS HEURISTICA")
            L = [  (k,m,p) for k in range(1,n1+1) for m in M[k] for p in D ]
            Au = Auto()
            costo = Au[-1]["Obj"]
            tiempo = Au[-1]["Time"]
            Au = Au[0]
            
            #Autos
            Autos = { (k,m): (d,Au[(k,m,d)]) for(k,m,d) in Au.keys() }
            Autos

            #Crear K Para X

            X_D = [(k, m, i, d) for (k,m,d) in Au.keys()  for i in Corte_Entrada_Nodo(d,Ad)   ]
            X_A = [(k, m, i, j) for (k,m) in Autos.keys()  for (i,j) in A   ]
            X_O = [(k, m, j[0], j[1]) for (k,m) in Autos.keys() for j in Ao if "O"+str(k) == j[0]  ]
            K = X_O + X_A + X_D

            #Crear R Para y

            R = [(k,m,d) for (k,m,d) in Au.keys()  ] + [(k,m,i) for (k,m) in Autos.keys() for i in range(1,n+1)]
            
            
            print("HEURISTICA ASIGANCIÓN")
            Ruta, Pro, Par =  Heuritic(time = 2000)
            
            Par["Time"] = Par["Time"] + tiempo
            
            DatosInstancias6.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta, Pro, Par] })

<span style="color: #ff0000">60,23 y 18</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 797 rows, 1476 columns and 2952 nonzeros
Model fingerprint: 0x762a6977
Variable types: 0 continuous, 1476 integer (738 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 797 rows, 1476 columns, 2952 nonzeros
Variable types: 0 continuous, 1476 integer (738 binary)

Root relaxation: objective 8.022521e+03, 875 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8022.52101    0   20          - 8022.52101      -     -    0s
H    0     0                    1

    11    16 19824.8527    3 1124          - 19824.2000      -   592   89s
    15    20 19828.3146    4  929          - 19824.8541      -   754   93s
    19    24 19825.7862    4  863          - 19825.3021      -   730   96s
    27    32 19834.5389    5  848          - 19827.9854      -   718  103s
    31    36 19835.2901    6  893          - 19827.9854      -   725  105s
    35    40 19838.7800    7  793          - 19827.9854      -   729  110s
    44    49 19839.5539    8  648          - 19827.9854      -   709  115s
    52    58 19844.6325    9  669          - 19827.9854      -   705  122s
    57    63 19844.5446   10  706          - 19827.9854      -   698  127s
    62    69 19845.8623   11  708          - 19827.9854      -   710  130s
    72    81 19848.7375   13  633          - 19827.9854      -   698  137s
    80    88 19855.1685   14  599          - 19827.9854      -   682  141s
    87    95 19851.9813   15  619          - 19827.9854      -   669  145s
   101   109 19853.5186  

  5164  4765 20157.0525  320 1495          - 19853.5379      -   269 1540s
  5166  4766 21209.7279  779 1487          - 19853.5866      -   269 1553s
  5167  4767 19895.8735   62 1373          - 19853.8257      -   269 1568s
  5168  4767 20763.5335  801 1382          - 19853.9680      -   269 1571s
  5171  4769 20120.7174  285 1458          - 19854.1019      -   269 1575s
  5172  4770 20780.3076  773 1507          - 19854.1065      -   269 1586s
  5173  4771 20829.1178  816 1408          - 19855.0403      -   269 1602s
  5175  4772 19855.4581    9 1482          - 19855.4581      -   269 1606s
  5178  4774 22784.2108 1155 1529          - 19855.5506      -   268 1620s
  5179  4775 19862.9306   22 1346          - 19856.6671      -   268 1639s
  5180  4775 19875.2278   33 1398          - 19856.8527      -   268 1641s
  5182  4777 21750.0082  952 1428          - 19856.9119      -   268 1653s
  5183  4777 20085.0479  264 1484          - 19857.8065      -   268 1673s
  5184  4778 19974.0464  

<span style="color: #ff0000">65,15 y 9</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 259 rows, 450 columns and 900 nonzeros
Model fingerprint: 0xa2c25c94
Variable types: 0 continuous, 450 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 259 rows, 450 columns, 900 nonzeros
Variable types: 0 continuous, 450 integer (225 binary)

Root relaxation: objective 5.495467e+03, 278 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5495.46667    0   12          - 5495.46667      -     -    0s
H    0     0                    7690.00

   305   325 10769.3972   45  695          - 10729.4378      -   440  164s
   324   349 10771.5667   47  714          - 10729.4378      -   441  168s
   348   372 10773.7406   50  714          - 10729.4378      -   439  174s
   371   395 10780.3099   55  812          - 10729.4378      -   441  185s
   394   421 10783.7233   58  715          - 10729.4378      -   439  190s
   420   445 10793.2852   61  716          - 10729.4378      -   440  196s
   444   472 10795.7818   66  688          - 10729.4378      -   441  202s
   471   502 10800.9370   71  707          - 10729.4378      -   445  208s
   501   536 10806.1765   74  615          - 10729.4378      -   447  215s
   535   571 10810.6715   78  654          - 10729.4378      -   446  222s
   570   609 10816.0931   85  638          - 10729.4378      -   445  229s
   608   657 10821.9422   91  623          - 10729.4378      -   444  237s
   657   704 10824.2865   98  658          - 10729.4378      -   437  245s
   704   748 10836.7975  

  3656  3508 11362.4965  426 1926          - 10783.0870      -   270 1107s
  3657  3509 11768.9545  704 1908          - 10783.2428      -   270 1115s
  3660  3511 10869.1259  136 1920          - 10783.3368      -   270 1124s
  3661  3511 10859.7613  119 1989          - 10783.4029      -   269 1137s
  3662  3512 11033.9356  252 1976          - 10783.4313      -   269 1145s
  3663  3513 10980.5479  337 1898          - 10783.4907      -   269 1150s
  3665  3514 10969.3607  249 1921          - 10783.8169      -   269 1158s
  3668  3517 10856.0123  115 1115          - 10783.8169      -   283 1163s
  3670  3518 10843.9917   89  503          - 10783.8169      -   283 1178s
  3671  3519 11523.9044  453  798          - 10783.8169      -   283 1193s
  3672  3520 10787.3465   40 1029          - 10783.8169      -   283 1204s
  3673  3520 11420.7528  407 1180          - 10783.8169      -   283 1210s
  3674  3521 10783.8169   23 1403          - 10783.8169      -   282 1217s
  3675  3522 11019.4502  

El auto 1 del depósito 7 recoge 2.0 en 4
El auto 1 del depósito 7 recoge 22.0 en 24
El auto 1 del depósito 7 recoge 22.0 en 58
El auto 1 del depósito 8 recoge 5.0 en 15
El auto 1 del depósito 8 recoge 23.0 en 29
El auto 1 del depósito 8 recoge 22.0 en 38
El auto 2 del depósito 8 recoge 4.0 en 6
El auto 2 del depósito 8 recoge 23.0 en 9
El auto 2 del depósito 8 recoge 23.0 en 36
El auto 1 del depósito 9 recoge 22.0 en 10
El auto 1 del depósito 9 recoge 22.0 en 17
El auto 1 del depósito 9 recoge 23.0 en 43
El auto 1 del depósito 9 recoge 23.0 en 55
El auto 1 del depósito 12 recoge 21.0 en 12
El auto 1 del depósito 12 recoge 3.0 en 16
El auto 1 del depósito 12 recoge 22.0 en 47
El auto 1 del depósito 12 recoge 23.0 en 56
El auto 1 del depósito 12 recoge 23.0 en 65
El auto 2 del depósito 12 recoge 23.0 en 22
El auto 2 del depósito 12 recoge 23.0 en 33
El auto 2 del depósito 12 recoge 23.0 en 41
El auto 2 del depósito 12 recoge 22.0 en 57
El auto 3 del depósito 12 recoge 23.0 en 18
El auto 

In [31]:
for i in DatosInstancias6.keys():
    DatosInstancias4[i] = DatosInstancias6[i]

## INTANCIAS A LAS CUALES NO SE OBTUVO SOLUCIÓN FACTIBLE NI EN 600SEGUNDOS NI EN 900SEGUNDOS NI EN 2000SEGUNDOS

In [24]:
import pandas as pd
from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal - Copy.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {65: {25: [19]} }

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias7 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }
                        
           
            
            print("AUTOS HEURISTICA")
            L = [  (k,m,p) for k in range(1,n1+1) for m in M[k] for p in D ]
            Au = Auto()
            costo = Au[-1]["Obj"]
            tiempo = Au[-1]["Time"]
            Au = Au[0]
            
            #Autos
            Autos = { (k,m): (d,Au[(k,m,d)]) for(k,m,d) in Au.keys() }
            Autos

            #Crear K Para X

            X_D = [(k, m, i, d) for (k,m,d) in Au.keys()  for i in Corte_Entrada_Nodo(d,Ad)   ]
            X_A = [(k, m, i, j) for (k,m) in Autos.keys()  for (i,j) in A   ]
            X_O = [(k, m, j[0], j[1]) for (k,m) in Autos.keys() for j in Ao if "O"+str(k) == j[0]  ]
            K = X_O + X_A + X_D

            #Crear R Para y

            R = [(k,m,d) for (k,m,d) in Au.keys()  ] + [(k,m,i) for (k,m) in Autos.keys() for i in range(1,n+1)]
            
            
            print("HEURISTICA ASIGANCIÓN")
            Ruta, Pro, Par =  Heuritic(time=4200)
            
            Par["Time"] = Par["Time"] + tiempo
            
            DatosInstancias7.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta, Pro, Par] })

<span style="color: #ff0000">65,25 y 19</span> 

AUTOS HEURISTICA
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1179 rows, 2204 columns and 4408 nonzeros
Model fingerprint: 0xa3eae584
Variable types: 0 continuous, 2204 integer (1102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 1179 rows, 2204 columns, 4408 nonzeros
Variable types: 0 continuous, 2204 integer (1102 binary)

Root relaxation: objective 9.081078e+03, 1258 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9081.07767    0   25          - 9081.07767      -     -    0s
H    0     0                

     0     0 20008.1854    0 1371          - 20008.1854      -     -   75s
     0     0 20008.5330    0 1416          - 20008.5330      -     -   78s
     0     0 20008.6315    0 1425          - 20008.6315      -     -   79s
     0     0 20008.6523    0 1467          - 20008.6523      -     -   80s
     0     0 20008.9807    0 1443          - 20008.9807      -     -   83s
     0     0 20009.3248    0 1402          - 20009.3248      -     -   84s
     0     0 20009.8225    0 1400          - 20009.8225      -     -   85s
     0     0 20009.8389    0 1456          - 20009.8389      -     -   86s
     0     0 20010.1275    0 1434          - 20010.1275      -     -   88s
     0     0 20010.1863    0 1422          - 20010.1863      -     -   89s
     0     0 20010.2129    0 1434          - 20010.2129      -     -   90s
     0     0 20010.5780    0 1530          - 20010.5780      -     -   93s
     0     0 20010.7054    0 1518          - 20010.7054      -     -   94s
     0     0 20010.7327  

  5157  5068 21648.8604  890 1451          - 20018.6653      -   430 2678s
  5158  5069 20276.8410  225 1462          - 20018.9376      -   430 2682s
  5159  5069 21460.5614  770 1475          - 20019.1309      -   430 2686s
  5161  5071 20063.3890   52 1535          - 20019.2001      -   430 2701s
  5162  5071 20044.6237   17 1364          - 20024.5172      -   430 2733s
  5163  5072 20338.1331  296 1522          - 20026.0167      -   430 2739s
  5164  5073 22210.3003 1062 1500          - 20026.4757      -   430 2742s
  5165  5073 21513.3586  825 1475          - 20026.7896      -   430 2745s
  5168  5075 20553.5542  430 1594          - 20026.9302      -   429 2760s
  5169  5076 22368.7499 1073 1611          - 20028.9807      -   429 2781s
  5170  5077 20401.7864  347 1662          - 20030.5888      -   429 2788s
  5171  5077 20166.8779  189 1590          - 20031.4739      -   429 2795s
  5172  5078 21513.3586  825 1627          - 20031.6356      -   429 2800s
  5174  5079 20355.8923  

el auto 1 del deposito 5 visita 35
el auto 3 del deposito 5 visita 2
el auto 3 del deposito 5 visita 13
el auto 3 del deposito 5 visita 44
el auto 4 del deposito 5 visita 1
el auto 4 del deposito 5 visita 20
el auto 4 del deposito 5 visita 34
el auto 4 del deposito 5 visita 59
el auto 1 del deposito 6 visita 15
el auto 1 del deposito 6 visita 48
el auto 1 del deposito 6 visita 61
el auto 3 del deposito 7 visita 33
el auto 4 del deposito 7 visita 31
el auto 4 del deposito 7 visita 32
el auto 4 del deposito 7 visita 51
el auto 1 del deposito 8 visita 17
el auto 1 del deposito 8 visita 37
el auto 1 del deposito 8 visita 55
el auto 2 del deposito 8 visita 8
el auto 2 del deposito 8 visita 9
el auto 2 del deposito 8 visita 36
el auto 2 del deposito 8 visita 63
el auto 3 del deposito 9 visita 4
el auto 3 del deposito 9 visita 11
el auto 3 del deposito 9 visita 16
el auto 3 del deposito 9 visita 34
el auto 4 del deposito 9 visita 46
el auto 1 del deposito 10 visita 6
el auto 1 del deposito 10

In [32]:
for i in DatosInstancias7.keys():
    DatosInstancias4[i] = DatosInstancias7[i]

## PRESENTACIÓN DE RESUTADOS

In [33]:
for i in DatosInstancias4.keys():
    print("____"*20)
    print("INSTANCIA {}".format(i))
    print(DatosInstancias4[i][-1] )

________________________________________________________________________________
INSTANCIA (5, 11, 17)
{'Obj': 18906.0, 'GAP': 0.0, 'Time': 0.21551513671875}
________________________________________________________________________________
INSTANCIA (5, 14, 17)
{'Obj': 18906.0, 'GAP': 0.0, 'Time': 0.22120094299316406}
________________________________________________________________________________
INSTANCIA (5, 17, 2)
{'Obj': 7004.0, 'GAP': 0.0, 'Time': 0.17165374755859375}
________________________________________________________________________________
INSTANCIA (5, 18, 6)
{'Obj': 9728.0, 'GAP': 0.0, 'Time': 0.17377281188964844}
________________________________________________________________________________
INSTANCIA (10, 11, 10)
{'Obj': 9531.0, 'GAP': 0.0, 'Time': 0.7371501922607422}
________________________________________________________________________________
INSTANCIA (10, 15, 4)
{'Obj': 8573.0, 'GAP': 0.0, 'Time': 1.859670639038086}
_____________________________________________

## GUARDAR INDICADORES EN CADA INSTANCIA

In [50]:
Heuristica = pd.DataFrame()
Heuristica = Heuristica.reindex(DatosInstancias4.keys())


Heuristica[("HEURISTICA 1","OBJ")] = pd.Series([DatosInstancias4[i][-1]["Obj"] for i in DatosInstancias4.keys()], index = DatosInstancias4.keys())
Heuristica[("HEURISTICA 1","GAP")] = pd.Series([DatosInstancias4[i][-1]["GAP"] for i in DatosInstancias4.keys()], index = DatosInstancias4.keys())
Heuristica[("HEURISTICA 1","Time")] = pd.Series([DatosInstancias4[i][-1]["Time"] for i in DatosInstancias4.keys()], index = DatosInstancias4.keys())

In [72]:
Data = pd.read_excel("RESULTADOS_INSTANCIAS_FINAL.xlsx", index_col=[0,1,2] , header= [0,1])

In [76]:
Data1 = pd.concat([Data, Heuristica], axis=1)

In [78]:
Data1.to_excel("Instancias_Soluciones_Final.xlsx")

 ## GUARDAR SOLUCIONES EN *.txt

In [88]:
Soluciones = {}
for i in DatosInstancias4.keys():
    Soluciones[i] = tuple(DatosInstancias4[i][-3:])

In [91]:
# open file for writing
f = open("SolHeu.txt","w")

# write file
f.write( str(Soluciones) )

# close file
f.close()